In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

# Code from "ml_death_star"

In [76]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder


# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy="constant")

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# Bundle preprocessing for numerical and categorical data
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('numerical', numerical_transformer, numerical_cols),
#         ('categorical', categorical_transformer, categorical_cols)
#     ])


class Preprocessor:
    def __init__(self, numerical_cols, categorical_cols):
        self.pipeline = ColumnTransformer(transformers=[
            ('numerical', numerical_transformer, numerical_cols),
            ('categorical', categorical_transformer, categorical_cols)]
        )

    def __repr__(self):
        return f"Preprocessor with several steps" # TODO WHAT


class ModelPipeline(Preprocessor):
    def __init__(self, numerical_cols, categorical_cols, model):
        super().__init__(numerical_cols, categorical_cols)
        self.pipeline = Pipeline(steps=[
            ("preprocessor", Preprocessor(numerical_cols, categorical_cols)),
            ("model", model)
        ])

In [77]:
data = pd.read_csv("data/melb_data.csv")
y = data.Price
X = data.drop(['Price'], axis=1)

X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, random_state=0, test_size=0.2)

#"Cardinality" means the number of unique values in a column
#Select categorical columns with relatively low cardinality (convenient but arbitrary)

categorical_cols = [col_name for col_name in X_train_full.columns if all((X_train_full[col_name].nunique() < 10,
                                                                         X_train_full[col_name].dtype == "object"))]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [78]:
X_valid

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
8505,h,SP,Western Metropolitan,4,8.0,3016.0,4.0,2.0,2.0,450.0,190.0,1910.0,-37.86100,144.89850,6380.0
5523,h,S,Western Metropolitan,2,6.6,3011.0,2.0,1.0,0.0,172.0,81.0,1900.0,-37.81000,144.88960,2417.0
12852,h,SP,Western Metropolitan,3,10.5,3020.0,3.0,1.0,1.0,581.0,NaN,NaN,-37.76740,144.82421,4217.0
4818,t,PI,Southern Metropolitan,3,4.5,3181.0,2.0,2.0,1.0,128.0,134.0,2000.0,-37.85260,145.00710,7717.0
12812,h,S,Northern Metropolitan,3,8.5,3044.0,3.0,2.0,2.0,480.0,NaN,NaN,-37.72523,144.94567,7485.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2664,u,PI,Western Metropolitan,2,6.4,3011.0,2.0,1.0,1.0,47.0,35.0,2013.0,-37.80140,144.89590,7570.0
8513,h,SP,Western Metropolitan,4,8.0,3016.0,4.0,2.0,4.0,551.0,NaN,NaN,-37.85790,144.87860,6380.0
12922,h,S,Eastern Metropolitan,3,10.8,3105.0,3.0,1.0,1.0,757.0,NaN,NaN,-37.78094,145.10131,4480.0
10761,h,S,Western Metropolitan,4,6.2,3039.0,4.0,1.0,3.0,478.0,152.0,1925.0,-37.76421,144.90571,6232.0


In [79]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.8272,4217.0


In [80]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

pipeline = ModelPipeline(numerical_cols=numerical_cols, categorical_cols=categorical_cols, model=model)


TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'Preprocessor with several steps' (type <class '__main__.Preprocessor'>) doesn't